# Text Wrangling and Regex

Working with text: applying string methods and regular expressions

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import zipfile
import pandas as pd
import os
import re

## Demo 1: Canonicalizing County Names

Load both **county_and_state.csv** and **county_and_population.csv**

In [3]:
# display both frames
#code Here
df1=pd.read_csv("/content/drive/MyDrive/DataScience_with_KS/Pandas Work/Text wrangling and regex/county_and_state.csv")
df2=pd.read_csv("/content/drive/MyDrive/DataScience_with_KS/Pandas Work/Text wrangling and regex/county_and_population.csv")


In [4]:
df1.head()

,County,State
0,De Witt County,IL
1,Lac qui Parle County,MN
2,Lewis and Clark County,MT
3,St John the Baptist Parish,LS


In [5]:
df2.head()

,County,Population
0,DeWitt,16798
1,Lac Qui Parle,8067
2,Lewis & Clark,55716
3,St. John the Baptist,43044


Both of these DataFrames share a "County" column. Unfortunately, formatting differences mean that we can't directly merge the two DataFrames using the "County"s.

In [6]:
#merge both frames usding pd.merge
#The output would only show the header. We will resolve it later on.
pd.merge(df1,df2)


,County,State,Population


To address this, we can **canonicalize** the "County" string data to apply a common formatting.

In [7]:
def canonicalize_county(county_series):
  return (county_series
          .str.lower() # lowercase
          .str.replace(" ","") # remove space
          .str.replace("&","") # replace &
          .str.replace(".","")  # remove dot
          .str.replace("county","")  # remove "county"
          .str.replace("parish",""))  # remove "parish"

Apply canonicalize_county on 'County' columns in both frames.

In [8]:
# Code Here
df1['County'] = canonicalize_county(df1['County'])
df2['County'] = canonicalize_county(df2['County'])

Now, the merge works as expected!

In [9]:
# code Here
#now merge df1 and df2
pd.merge(df1,df2,on="County",how="inner")

,County,State,Population
0,dewitt,IL,16798
1,lacquiparle,MN,8067
2,stjohnthebaptist,LS,43044


## Demo 2: Extracting Log Data

Load log.txt

In [10]:
from posixpath import sep
#code Here
import json
        # Process each line as needed
file_path="/content/drive/MyDrive/DataScience_with_KS/Pandas Work/Text wrangling and regex/log.txt"
with open(file_path,"r") as f:
  data=f.readlines()

#Hint: the file will be read almost the same way as we read the json file.
#Use the file.readlines function to read the lines in the txt file


Suppose we want to extract the day, month, year, hour, minutes, seconds, and timezone. Looking at the data, we see that these items are not in a fixed position relative to the beginning of the string. That is, slicing by some fixed offset isn't going to work.

Instead, we'll need to use some more sophisticated thinking. Let's focus on only the first line of the file.

In [11]:
# code here
data[0]

'169.237.46.168 - - [26/Jan/2014:10:47:58 -0800] "GET /stat141/Winter04/ HTTP/1.1" 200 2585 "http://anson.ucdavis.edu/courses/"\n'

Apply string functions of python to extract date from first entry in log file.

In [12]:
# you might take help from the slides
line=data[0]
                                        # find the text enclosed in square brackets
sqr_data=line.split('[')[1].split(']')[0]
sqr_data
                                        # split up the date/month/year
date,month,rest=sqr_data.split('/')

                                        # split up the hour:minute:second
year,hour,minute,rest=rest.split(':')
                                        # split the timezone after the blank space
sec,time_zone=rest.split(' -')

This worked, but felt fairly "hacky" – the code above isn't particularly elegant. A much more sophisticated but common approach is to extract the information we need using a *regular expression*.


# Regular Expressions


## String Extraction with Regex

Python `re.findall` returns a list of all extracted matches: extract numbers from given string.

In [13]:
import re

text = "My social security number is 123-45-6789 bro, or actually maybe it’s 321-45-6789.";
# code Here
pattern=r'\d+\-\d+\-\d+'
re.findall(pattern,text)
#Expected output:  ['123-45-6789', '321-45-6789']

['123-45-6789', '321-45-6789']

<br/>

Now, let's see vectorized extraction in `pandas`:

 `.str.findall` returns a `Series` of lists of all matches in each record.

In [14]:
data = ['987-65-4321','forty','123-45-6789 bro or 321-45-6789','999-99-9999']
# code here to convert the above data into a dataframe

Find all entries containing numbers in dataframe.

In [15]:
# -> Series of lists
# code Here
data=pd.DataFrame(data)
pattern=r'(\d+)\-(\d+)\-(\d+)'
ser=data[0].str.findall(pattern)
ser
#Expected Output:
#  0                   [987, 65, 4321]
#  1                                []
#  2    [123, 45, 6789, 321, 45, 6789]
#  3                   [999, 99, 9999]
#  Name: SSN, dtype: object

,0
0,"[(987, 65, 4321)]"
1,[]
2,"[(123, 45, 6789), (321, 45, 6789)]"
3,"[(999, 99, 9999)]"


## Extraction Using Regex Capture Groups

The Python function `re.findall`, in combination with parentheses returns specific substrings (i.e., **capture groups**) within each matched string, or **match**.

In [16]:
text = """I will meet you at 08:30:00 pm tomorrow"""
# code here
pattern=r"(\d+):(\d+):(\d+)"
matches=re.findall(pattern,text)

In [17]:
# the three capture groups in the first matched string
hour, minute, second = matches[0]
minute

'30'

<br/>

In `pandas`, we can use `.str.extract` to extract each capture group of **only the first match** of each record into separate columns.

In [18]:
# back to SSNs
ssn=data
ssn=pd.DataFrame(ssn)
ssn

,0
0,987-65-4321
1,forty
2,123-45-6789 bro or 321-45-6789
3,999-99-9999


In [19]:
# Will extract the first match of all groups
# code here
pattern=r'(\d+-\d+-\d+)'
ssn_extract=ssn[0].str.extract(pattern)
ssn_extract

,0
0,987-65-4321
1,NaN
2,123-45-6789
3,999-99-9999


Alternatively, `.str.extractall` extracts **all matches** of each record into separate columns. Rows are then MultiIndexed by original record index and match index.

In [57]:
# -> DataFrame, one row per match
# code Here
pattern=r'(\d+-\d+-\d+)'
ssn_extractall=ssn[0].str.extractall(pattern)
ssn_extractall

0
  match             
0 0      987-65-4321
2 0      123-45-6789
  1      321-45-6789
3 0      999-99-9999

## Canonicalization with Regex

In regular Python, canonicalize with `re.sub` (standing for "substitute"):

In [21]:
text = '<div><td valign="top"><div>Moo</td></div>'

#code here to extract the word Moo
pattern=r"<[^>]+>"
moo_value=re.sub(pattern,"",text)
moo_value

'Moo'

<br/>

In `pandas`, canonicalize with `Series.str.replace`.

In [22]:
# example dataframe of strings, convert in dataframe
df_html = ['<div><td valign="top">Moo</td></div>',
                   '<a href="http://ds100.org">Link</a>',
                   '<b>Bold text</b>']
df_html

['<div><td valign="top">Moo</td></div>',
 '<a href="http://ds100.org">Link</a>',
 '<b>Bold text</b>']

In [23]:
# Series -> Series
#Extract only words from the above given df_html like Moo, Link and Bold text
pattern=r"<[^>]+>"
values=[]
for i in df_html:
  values.append(re.sub(pattern,"",i))
values


['Moo', 'Link', 'Bold text']


# Revisiting Text Log Processing using Regex

### Python `re` version

In [60]:
file_path="/content/drive/MyDrive/DataScience_with_KS/Pandas Work/Text wrangling and regex/log.txt"
with open(file_path,"r") as f:
  log_lines=f.readlines()
# lines=pd.DataFrame(log_lines)
# line = log_lines[0]
# display(line)
# pattern=r"(\d+)/(.+)/(\d+):(\d+):(\d+):(\d+) -(\d+)"
# date,month,year,hour,minute,sec,time_zone=lines[0].str.extract(pattern)

'169.237.46.168 - - [26/Jan/2014:10:47:58 -0800] "GET /stat141/Winter04/ HTTP/1.1" 200 2585 "http://anson.ucdavis.edu/courses/"\n'

0

In [36]:
file_path="/content/drive/MyDrive/DataScience_with_KS/Pandas Work/Text wrangling and regex/log.txt"
with open(file_path,"r") as f:
  log_lines=f.readlines()

line = log_lines[0]
display(line)
# code here
df=pd.DataFrame()
Day=[]
Month=[]
Year=[]
Hour=[]
Minute=[]
Second=[]
Time_Zone=[]

for i in log_lines:

  sqr_data=i.split('[')[1].split(']')[0]


  date,month,rest=sqr_data.split('/')


  year,hour,minute,rest=rest.split(':')

  sec,time_zone=rest.split(' -')
  Day.append(date)
  Month.append(month)
  Year.append(year)
  Hour.append(hour)
  Minute.append(minute)
  Second.append(sec)
  Time_Zone.append(time_zone)

'169.237.46.168 - - [26/Jan/2014:10:47:58 -0800] "GET /stat141/Winter04/ HTTP/1.1" 200 2585 "http://anson.ucdavis.edu/courses/"\n'

### `pandas` version

In [46]:
# code here
Time=pd.DataFrame({"Day":Day,"Month":Month,"Year":Year,"Hour":Hour,"Minute":Minute,"Second":Second,"Time_Zone":Time_Zone})
Time

,Day,Month,Year,Hour,Minute,Second,Time_Zone
0,26,Jan,2014,10,47,58,0800
1,2,Feb,2005,17,23,6,0800
2,3,Feb,2006,10,18,37,0800


Option 1: `Series.str.findall`

In [50]:
# code here

<br/>

Option 2: `Series.str.extractall`

In [44]:
# code here
Time.head()

,Day,Month,Year,Hour,Minute,Second,Time_Zone
0,26,Jan,2014,10,47,58,0800
1,2,Feb,2005,17,23,6,0800
2,3,Feb,2006,10,18,37,0800


Wrangling either of these two DataFrames into a nice format (like below) is left as an exercise for you!


||Day|Month|Year|Hour|Minute|Second|Time Zone|
|---|---|---|---|---|---|---|---|
|0|26|Jan|2014|10|47|58|-0800|
|1|2|Feb|2005|17|23|6|-0800|
|2|3|Feb|2006|10|18|37|-0800|


In [61]:
# your code here
...
# I have already extracted the values and stored then in a data frame .
Time

,Day,Month,Year,Hour,Minute,Second,Time_Zone
0,26,Jan,2014,10,47,58,0800
1,2,Feb,2005,17,23,6,0800
2,3,Feb,2006,10,18,37,0800
